# Loading the data

In [125]:
import pandas as pd
import numpy as np

df = pd.read_excel('mysore_city_data.xlsx')
df['day']=pd.to_datetime(df.Date,format="%Y-%m-%d").dt.day
df['month']=pd.to_datetime(df.Date,format="%Y-%m-%d").dt.month
df['year']=pd.to_datetime(df.Date,format="%Y-%m-%d").dt.year

df.drop(['Date'],axis=1,inplace=True)

In [126]:
print(df)

      Temperature  Humidity        Gas        CO   NH3  PM 2.5 (ug/m3)  PM10  \
0            31.0      64.0  45.000000  0.340000  47.0        5.000000     3   
1            31.0      63.0  48.000000  0.590000  28.0        5.000000     3   
2            31.0      63.0  58.000000  0.340000  29.0        5.000000     3   
3            31.0      63.0  48.000000  0.680000  31.0        7.000000     4   
4            31.0      63.0  42.000000  0.320000  26.0        7.000000     4   
...           ...       ...        ...       ...   ...             ...   ...   
4529         30.0      64.0  38.000000  0.530000  18.0       13.000000    10   
4530         30.0      64.0  42.000000  0.290000  37.0       13.000000    10   
4531         30.0      64.0  53.000000  0.220000  26.0       13.000000    10   
4532         30.0      64.0  49.000000  0.280000  37.0       13.000000    10   
4533         30.0      64.0  46.306294  0.444436  37.0       22.536731    10   

      day  month  year  
0      16     

# Just handling the missing values and not outliers

In [127]:
def fill_mean(data):#replacing null fields with the mean value of the respective filed
    null_fields = data.isna().sum()
    col = data.columns #storing the column names
    x = 0
    for i in null_fields:
        if i != 0:
            data = data.fillna({col[x]:data[col[x]].mean()})# replaces null field with mean of column values
        x += 1     
    return data

df = fill_mean(df)

# Handling outliers

In [128]:
import pandas as pd
import numpy as np

## handling outliers

def detect_outliers_iqr(data):
    outliers = []
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers

def collect_outliers_iqr(data):
    outliers_detected_iqr = {}
    for i in data.columns:
        outliers = detect_outliers_iqr(data[i])
        outliers_detected_iqr[i] = outliers
    return outliers_detected_iqr


def floor_clapp_outliers(data, outliers):
    for i, j in outliers.items():
        if len(outliers[i]) != 0:
            IQR = data[i].quantile(0.75) - data[i].quantile(0.25)
            lower_bridge = data[i].quantile(0.25) - (IQR*1.5)
            upper_bridge = data[i].quantile(0.75) + (IQR*1.5)
            data.loc[data[i] > upper_bridge, i] = upper_bridge
            data.loc[data[i] < lower_bridge, i] = lower_bridge
    return data

outliers = collect_outliers_iqr(df)
df = floor_clapp_outliers(df, outliers)

In [129]:
df.columns

Index(['Temperature', 'Humidity', 'Gas', 'CO', 'NH3', 'PM 2.5 (ug/m3)', 'PM10',
       'day', 'month', 'year'],
      dtype='object')

# Separating the independent and dependent variables(features)

In [130]:
X = df[['Temperature', 'Humidity', 'Gas', 'CO', 'NH3','PM10', 'day',
       'month', 'year']]
y = df['PM 2.5 (ug/m3)']

# Splitting the data into training and testing parts, 65% of the dataset will be used for training the model, whereas 35% dataset will be used for testing the accuracy of the model.

In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=0)

print('X_train : ',X_train.shape,' X_test : ',X_test.shape)
print('y_train : ',y_train.shape,'    y_test : ',y_test.shape)

X_train :  (2947, 9)  X_test :  (1587, 9)
y_train :  (2947,)     y_test :  (1587,)


# Testing with different regression models

In [132]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
import math
scores = {}

## 1. Using Linear Regression

## 2. Using Ridge Regression

In [133]:
%%time

from sklearn.linear_model import Ridge

ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X_train,y_train)
y_pred = ridge_regressor.predict(X_test)

scores['RidgeRegression '] = round((r2_score(y_test,y_pred)*100),2)

print("Ridge Regression_________________________")
print()
print('R2 score            : ',r2_score(y_test,y_pred))
print('mean Absolute Error : ',math.sqrt(mean_squared_error(np.array(y_test),y_pred)))
print("Accuracy            :  {:.2f}".format(r2_score(y_test,y_pred)*100),"%")
print('-----------------------------------------')

C:\Users\Rituraj\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.015975e-21
  overwrite_a=True).T
C:\Users\Rituraj\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.994785e-21
  overwrite_a=True).T
C:\Users\Rituraj\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.881741e-21
  overwrite_a=True).T
C:\Users\Rituraj\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.917357e-21
  overwrite_a=True).T


Ridge Regression_________________________

R2 score            :  0.9475918965186101
mean Absolute Error :  2.189535070640946
Accuracy            :  94.76 %
-----------------------------------------
Wall time: 539 ms


## 4. Using Decision Tree Regression

In [134]:
%%time

from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=4, min_samples_leaf=0.1, random_state=3)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
       
scores['DecisionTreeReg.'] = round((r2_score(y_test,y_pred)*100),2)

print("Decision Tree Regression_________________")
print()
print('R2 score            : ',r2_score(y_test,y_pred))
print('mean Absolute Error : ',math.sqrt(mean_squared_error(np.array(y_test),y_pred)))
print("Accuracy            :  {:.2f}".format(r2_score(y_test,y_pred)*100),"%")
print('-----------------------------------------')

Decision Tree Regression_________________

R2 score            :  0.9192855404146786
mean Absolute Error :  2.7172422299229497
Accuracy            :  91.93 %
-----------------------------------------
Wall time: 7.98 ms


## 5. Using Random Forest

In [135]:
%%time

from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor(n_estimators = 150,random_state = 0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_pred=y_pred.round(decimals=3)

scores['RandomForestReg.'] = round((r2_score(y_test,y_pred)*100),2)

print("Random Forest Regression_________________")
print()
print('R2 score            : ',r2_score(y_test,y_pred))
print('mean Absolute Error : ',math.sqrt(mean_squared_error(np.array(y_test),y_pred)))
print("Accuracy            :  {:.2f}".format(r2_score(y_test,y_pred)*100),"%")
print('-----------------------------------------')

Random Forest Regression_________________

R2 score            :  0.9681089121069325
mean Absolute Error :  1.7079979736548776
Accuracy            :  96.81 %
-----------------------------------------
Wall time: 983 ms


## 6. Using K Nearest Neighbour (KNN)

In [136]:
%%time

from sklearn import neighbors
from math import sqrt

params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(X_train,y_train)
N = model.best_params_
N = N['n_neighbors']
model = neighbors.KNeighborsRegressor(n_neighbors = N)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
        
scores['KNNregression   '] = round((r2_score(y_test,y_pred)*100),2)

print("K Nearest Neighbour Regression___________")
print()
print('R2 score            : ',r2_score(y_test,y_pred))
print('mean Absolute Error : ',math.sqrt(mean_squared_error(np.array(y_test),y_pred)))
print("Accuracy            :  {:.2f}".format(r2_score(y_test,y_pred)*100),"%")
print('-----------------------------------------')

K Nearest Neighbour Regression___________

R2 score            :  0.9503023624934277
mean Absolute Error :  2.1321637522919703
Accuracy            :  95.03 %
-----------------------------------------
Wall time: 1.03 s


# All Scores in different models

In [137]:
count = 0
print('     Model               Accuracy')
print()
for i,j in scores.items():
    count +=1
    print(count,'. ',i,'->',j,'%')
    print()

     Model               Accuracy

1 .  RidgeRegression  -> 94.76 %

2 .  DecisionTreeReg. -> 91.93 %

3 .  RandomForestReg. -> 96.81 %

4 .  KNNregression    -> 95.03 %



### From above, we can conclude that for this project's requirement, Random Forest Regression (accuracy of 83 % ) is best suited.

In [138]:
## scores dictionary (without outlier handling)
scores

{'RidgeRegression ': 94.76,
 'DecisionTreeReg.': 91.93,
 'RandomForestReg.': 96.81,
 'KNNregression   ': 95.03}

In [139]:
## scores dictionary (with outlier handling)
scores

{'RidgeRegression ': 94.76,
 'DecisionTreeReg.': 91.93,
 'RandomForestReg.': 96.81,
 'KNNregression   ': 95.03}